In [ ]:
import cv2
import numpy as np
import controller_publisher_utils as utils
import time
from collections import Counter

IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
        


def warp_img(img,points,w,h,inv=False):
    
    pts1 = np.float32(points)                                   # points of starting image
    pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])                # points for the warp
    if inv:
        matrix = cv2.getPerspectiveTransform(pts2,pts1)
    else:
        matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarp = cv2.warpPerspective(img,matrix,(w,h))
    return imgWarp


def my_hist(hist):
    
    hist = cv2.cvtColor(hist, cv2.COLOR_BGR2GRAY)
    
    temp = [Counter(col) for col in zip(*hist)]
    white_count_col = [[i, temp[i][max(temp[i].keys())]] for i in range(len(temp))]
    avg = 0
    num_whites = 0
    for tpl in white_count_col:
        avg += tpl[0] * tpl[1]
        num_whites += tpl[1]
        
    avg = avg // num_whites
    
    #cv2.line(hist, (avg, hist.shape[0]), (avg, hist.shape[1]), 255, 1)
    #cv2.imshow("s", hist)
    
    return avg


def pipeline(img):
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = img_gray.shape
    img_blur = cv2.blur(img_gray, (3,3))
    img_canny = cv2.Canny(img_blur, 0, 200)
    points = utils.valTrackbars(IMAGE_WIDTH, IMAGE_HEIGHT)
    warp_im = warp_img(img_canny, points, h, w)
    filled_img = utils.fill_image(warp_im)
    _, hist = utils.getHistogram(filled_img, display_hist=True, minPercentage=0.1, region=5)    
    avg = my_hist(hist)
    
    treshold = 30
    
    cv2.line(hist, (avg, hist.shape[0]), (avg, hist.shape[1]), (0, 255, 255), 2)
    cv2.line(hist, (hist.shape[1]//2, 0), (hist.shape[1]//2, hist.shape[0]), (255, 0, 0), 2)
    cv2.line(hist, (hist.shape[1]//2 + treshold, 0), (hist.shape[1]//2 + treshold, hist.shape[0]), (0, 255, 0), 2)
    cv2.line(hist, (hist.shape[1]//2 - treshold, 0), (hist.shape[1]//2 - treshold, hist.shape[0]), (0, 255, 0), 2)
    
    return img, warp_im, filled_img, hist, warp_img(img, points, h, w)


def main():
    
    cap = cv2.VideoCapture('/home/eirik/ROS_car_project/output.avi')

    intialTracbarVals = [30, 300, 0, 480]  # width top, height top, width bottom, height bottom. Slider trackbar
    utils.initializeTrackbars(intialTracbarVals, IMAGE_WIDTH, IMAGE_HEIGHT)

    frameCounter = 0
    while True:
        # Restarts the video if end is reached
        frameCounter += 1
        if cap.get(cv2.CAP_PROP_FRAME_COUNT) == frameCounter:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frameCounter = 0

        _, img = cap.read()  # GET THE IMAGE
        img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))  # RESIZE
        img, _, filled_img, hist, warped = pipeline(img)
        cv2.imshow("video", warped)
        cv2.imshow("histogram", hist)
        
        both = cv2.addWeighted(warped, 0.5, hist, 0.5, 0.0)
        cv2.imshow("both", both)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):  # press q To exit
            break

    cap.release()
    cv2.destroyAllWindows()

main()